In [1]:
import os
from dataclasses import dataclass, asdict
from collections import defaultdict
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_sequence, pad_sequence
from tqdm import tqdm

In [25]:
DATA_DIR = "./RNA-Puzzles"
RESULTS_FILENAME = "filter-results.txt"
N = 100

In [26]:
def float_or_none(val: str):
    return None if val == "-" else float(val)


@dataclass
class NativeMotive:
    name: str
    segments_count: int
    residues_count: int
    nucleotide_ranges: list[str]
    sequences: list[str]

    @staticmethod
    def from_split_line(l: list[str]):
        return NativeMotive(
            name=l[0],
            segments_count=int(l[1]),
            residues_count=int(l[2]),
            nucleotide_ranges=list(map(lambda x: x.strip(), l[3].split(","))),
            sequences=list(map(lambda x: x.strip(), l[4].split(",")))
        )


@dataclass
class NucleotideTorAngles:
    chain: str
    residue_num: int
    name: str
    alpha: float
    beta: float
    gamma: float
    delta: float
    epsilon: float
    zeta: float
    chi: float

    @staticmethod
    def from_split_line(l: list[str]):
        return NucleotideTorAngles(
            chain=l[0],
            residue_num=int(l[1]),
            name=l[3],
            alpha=float_or_none(l[4]),
            beta=float_or_none(l[5]),
            gamma=float_or_none(l[6]),
            delta=float_or_none(l[7]),
            epsilon=float_or_none(l[8]),
            zeta=float_or_none(l[9]),
            chi=float_or_none(l[10])
        )
    
@dataclass
class MotivePrediction:
    name: str
    rmsd: float
    nucleotide_tor_angles: list[NucleotideTorAngles]
    

def get_tor_angles_from_file(filename: str) -> list[NucleotideTorAngles]:
    res = []
    with open(filename) as target_tor_file:
            next(target_tor_file)
            for line in target_tor_file:
                split_line = line.strip().split("\t")
                try:
                    res.append(asdict(NucleotideTorAngles.from_split_line(l=split_line)))
                except ValueError:
                    pass
    return res
    

In [27]:
puzzles = os.listdir(os.path.join(os.curdir, DATA_DIR))
puzzles

['pz01',
 'pz02',
 'pz03',
 'pz04',
 'pz05',
 'pz06',
 'pz07',
 'pz08',
 'pz09',
 'pz10']

In [28]:
native_motives = defaultdict(list[NativeMotive])

for puzzle in puzzles[:N]:
    with open(os.path.join(os.curdir, DATA_DIR, puzzle, RESULTS_FILENAME)) as results_file:
        for line in results_file:
            split_line = line.strip().split("\t")
            # TODO filter by segments count > 2/3?
            native_motives[puzzle].append(
                NativeMotive.from_split_line(l=split_line)
            )


In [29]:
motive_solutions = defaultdict(lambda: defaultdict(dict))
for puzzle in puzzles[:N]:
    for native_motive in native_motives[puzzle]:
        if not all(
                [
                    os.path.exists(os.path.join(os.curdir, DATA_DIR, puzzle, f'{native_motive.name}-rmsd.xml')),
                    os.path.exists(os.path.join(os.curdir, DATA_DIR, puzzle, f'{native_motive.name}.tor'))
                ]
        ):
            continue
        
        motive_solutions[puzzle][native_motive.name]['native_motive'] = native_motive
        motive_solutions[puzzle][native_motive.name]['target_tor_angles'] = get_tor_angles_from_file(os.path.join(os.curdir, DATA_DIR, puzzle, f'{native_motive.name}.tor'))
        motive_solutions[puzzle][native_motive.name]['predictions'] = []
        pred_files = list(filter(lambda x: x.endswith(".tor"), os.listdir(os.path.join(os.curdir, DATA_DIR, puzzle, native_motive.name))))
        for pred_file in pred_files:
            motive_solutions[puzzle][native_motive.name]['predictions'].append(
                MotivePrediction(
                    name=pred_file[:-4],
                    rmsd=-1,
                    nucleotide_tor_angles=get_tor_angles_from_file(os.path.join(os.curdir, DATA_DIR, puzzle, native_motive.name, pred_file))
                )
            )
        xml_tree = ET.parse(os.path.join(os.curdir, DATA_DIR, puzzle, f'{native_motive.name}-rmsd.xml'))
        prediction_scores = {s.find('description/filename').text[:-4]: float(s.find('score').text) for s in xml_tree.getroot().findall('structure')}
        
        for pred in motive_solutions[puzzle][native_motive.name]['predictions']:
            pred.rmsd = prediction_scores[pred.name]
        

In [ ]:
motive_solutions['pz01']['1_solution_0_rpr_B_18_U']

In [ ]:
"""
struktura słownika
motive_solutions{
    puzzle_name: {
        motive_name: {
            native_motive: NativeMotive() - klasa,
            target_tor_angles: list[dict(NucleotideTorAngles())] - lista kątów kolejnych nukleotydów z natywnego motywu,
            predictions: [
                MotivePrediction(
                name=...,
                rmsd=...,
                nucleotide_tor_angles: list[dict(NucleotideTorAngles())] - lista kątów kolejnych nukleotydów z danej predykcji motywu
                ),
                ...
            ] - lista wszystkich predykcji dla danego motywu
        }
    }
}
"""

In [57]:
dataset = []
for puzzle in motive_solutions:
    if puzzle in ('pz01', 'pz05'):
        set_type = 'test'
    else:
        set_type = 'train'

    for motive in motive_solutions[puzzle]:
        dataset.extend([dict(puzzle=puzzle, native_motive=motive_solutions[puzzle][motive]['native_motive'],
                                set=set_type, target_tor_angles=motive_solutions[puzzle][motive]['target_tor_angles'],
                                prediction_motive_name=motive_pred.name, rmsd=motive_pred.rmsd,
                                nucleotide_tor_angles=motive_pred.nucleotide_tor_angles)
                                for motive_pred in motive_solutions[puzzle][motive]['predictions']])

dataset = pd.DataFrame(dataset)
dataset

,puzzle,native_motive,set,target_tor_angles,prediction_motive_name,rmsd,nucleotide_tor_angles
0,pz01,"NativeMotive(name='1_solution_0_rpr_A_3_G', se...",test,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',...",1_bujnicki_1_rpr,4.643,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',..."
1,pz01,"NativeMotive(name='1_solution_0_rpr_A_3_G', se...",test,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',...",1_bujnicki_2_rpr,4.495,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',..."
2,pz01,"NativeMotive(name='1_solution_0_rpr_A_3_G', se...",test,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',...",1_bujnicki_3_rpr,3.862,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',..."
3,pz01,"NativeMotive(name='1_solution_0_rpr_A_3_G', se...",test,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',...",1_bujnicki_4_rpr,4.514,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',..."
4,pz01,"NativeMotive(name='1_solution_0_rpr_A_3_G', se...",test,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',...",1_bujnicki_5_rpr,4.617,"[{'chain': 'A', 'residue_num': 1, 'name': 'C',..."
...,...,...,...,...,...,...,...
34109,pz10,NativeMotive(name='10_0_solution_4LCK_rpr_B_64...,train,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'...",10_DING_5_rpr,2.963,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'..."
34110,pz10,NativeMotive(name='10_0_solution_4LCK_rpr_B_64...,train,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'...",10_DING_6_rpr,2.190,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'..."
34111,pz10,NativeMotive(name='10_0_solution_4LCK_rpr_B_64...,train,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'...",10_DING_7_rpr,2.374,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'..."
34112,pz10,NativeMotive(name='10_0_solution_4LCK_rpr_B_64...,train,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'...",10_DING_8_rpr,2.918,"[{'chain': 'B', 'residue_num': 47, 'name': 'C'..."


In [31]:
target_tor_angles_example_df = pd.DataFrame(dataset['target_tor_angles'].iloc[23])
nucleotide_tor_angles_example_df = pd.DataFrame(dataset['nucleotide_tor_angles'].iloc[23])
target_tor_angles_example_df

,chain,residue_num,name,alpha,beta,gamma,delta,epsilon,zeta,chi
0,A,1,C,NaN,176.850,44.959,91.876,-150.047,-80.396,-159.462
1,A,2,C,-56.496,169.031,44.356,74.273,-154.985,-76.230,-157.623
2,A,3,G,-69.487,177.430,52.512,77.981,-154.933,-75.264,-155.744
3,A,4,C,-60.263,170.927,42.313,81.796,-154.333,-69.952,-160.246
4,A,5,C,-77.439,179.135,59.432,76.937,-144.338,-78.585,-159.848
5,A,6,G,-62.513,161.032,58.492,81.831,-150.465,-79.457,-169.419
6,A,7,C,-47.744,165.734,42.144,87.830,-152.870,-72.211,-155.851
7,A,8,G,-64.037,175.025,49.996,81.011,NaN,NaN,-161.947
8,B,10,C,NaN,172.832,56.562,76.467,-151.416,-69.533,-162.329
9,B,11,A,-69.710,169.654,65.485,78.423,-158.423,-67.656,-169.968


In [59]:
# Split the data into training, validation, test sets
train_ds, test_ds = dataset[dataset.set == 'train'], dataset[dataset.set == 'test']
train_ds, valid_ds = train_test_split(train_ds, test_size=0.15, random_state=42)
valid_ds.set = 'valid'
print("Training dataset size:", train_ds.shape[0], train_ds.shape[0] / dataset.shape[0])
print("Validation dataset size:", valid_ds.shape[0], valid_ds.shape[0] / dataset.shape[0])
print("Test dataset size:", test_ds.shape[0], test_ds.shape[0] / dataset.shape[0])

Training dataset size: 24634 0.7221082253620215
Validation dataset size: 4348 0.12745500381075217
Test dataset size: 5132 0.15043677082722637


In [36]:
# Encoder-Decoder with Attention
class Encoder(nn.Module):
    def __init__(self, hidden_size=4, tor_features=11):
        super().__init__()
        self.rnn = nn.GRU(input_size=tor_features, hidden_size=hidden_size, bidirectional = True)
        
    def forward(self, src, src_len):
        packed_outputs = nn.utils.rnn.pack_padded_sequence(src, src_len, enforce_sorted=False)
        packed_outputs, hidden = self.rnn(packed_outputs)   
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)
        hidden = torch.cat([hidden[0,:, :], hidden[1,:,:]], dim=1)
        return outputs, hidden.unsqueeze(0)
    
class DecoderWithAttention(nn.Module):
    def __init__(self, hidden_size=8, tor_features=11):
        super().__init__()
        self.input_size = tor_features
        self.rnn = nn.GRU(input_size=hidden_size+tor_features, hidden_size=hidden_size)
        self.lin1 = nn.Linear(hidden_size, hidden_size//2)
        self.lin2 = nn.Linear(hidden_size//2, 1)
        self.fc = nn.Sequential(self.lin1, nn.ReLU(), self.lin2)
        
    
    def forward(self, input, hidden, encoder_outputs):
        
        # input = [batch size]
        # hidden = [batch size, hidden_size]
        # encoder_outputs = [src len, batch size, hidden_size]
        # mask = [batch size, src len]
        

        # context = [batch size, hidden_size]
        attention_weights = torch.bmm(hidden.permute(1, 0, 2), encoder_outputs.permute(1, 2, 0)).softmax(2).to(input.device)
        context = torch.bmm(attention_weights, encoder_outputs.permute(1, 0, 2)).squeeze(1).to(input.device)
        
        #Wejście do sieci do wektor kontekstu połączony z przetwarzanym słowem
        # print(input.shape, context.shape)
        rnn_input = torch.cat((input, context), dim = 1)
        
        # Wejście do sieci rekurencyjnej zawiera wymiar - długość przetwarzanej sekwencji
        # Konieczne jest więc "sztuczne" dodanie wymiaru na pierwszej (tj. zerowej) pozycji
        rnn_input = rnn_input.unsqueeze(0)
        #rnn_input = [1, batch size, word_embedding + hidden_size]
        
        _, hidden = self.rnn(rnn_input, hidden)
        
        prediction = self.fc(hidden.squeeze(0))
        
        return prediction, hidden

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = next(encoder.parameters()).device
        
    def forward(self, src, tgt, src_len, batch_size):       
        tgt_len = tgt.shape[0]
        
        # tensor to store decoder outputs
        outputs = torch.zeros(tgt_len, batch_size, 1).to(self.device)
        
        # Przetworzenie wejścia przez koder, kolejno uzyskane reprezentacje
        # kazdego słowa będą potrzebne do implementacji mechanizmu uwagi
        # Z kolei hidden posłuży do inicjalizacji stanu ukrytego dekodera
        encoder_outputs, hidden = self.encoder(src, src_len)
        
        #first input to the decoder is the <sos> tokens
        prev_target = tgt[0, :]
                
        for i in range(1, tgt_len):
            output, hidden = self.decoder(prev_target, hidden, encoder_outputs)
            outputs[i] = output
               
            # uczenie poprzez teaching forcing
            prev_target = tgt[i] # if self.training else output.argmax(1)
        return outputs

In [37]:
def preprocess_tor_data(tor_df: pd.DataFrame) -> pd.DataFrame:
    tor_df = tor_df.drop(['chain', 'residue_num'], axis=1)

    cols_to_normalize = [col for col in tor_df.columns if col != 'name']
    scaler = MinMaxScaler()
    tor_df[cols_to_normalize] = scaler.fit_transform(tor_df[cols_to_normalize])
    imp = SimpleImputer(missing_values=np.nan, strategy='median')
    tor_df[cols_to_normalize] = imp.fit_transform(tor_df[cols_to_normalize])

    # One hot encoding
    for nuc in 'ACUG':
        # tor_df[f'target_name_{nuc}'] = tor_df['target_name'].apply(str).str.contains(nuc).astype(int)
        tor_df[f'name_{nuc}'] = tor_df['name'].apply(str).str.contains(nuc).astype(int)

    tor_df = tor_df.drop(['name'], axis=1)
    return tor_df

def shuffle_batches(permutation, batch_size, x_len):
    batch_count = int(x_len // batch_size)
    result = [permutation[batch_idx*batch_size:(batch_idx+1)*batch_size] for batch_idx in range(batch_count)]
    return result

def create_batch(dataset, indxs, device):
    
    x1 = [torch.tensor(preprocess_tor_data(pd.DataFrame(dataset.iloc[i.item()]['target_tor_angles'])).
                       to_numpy()).to(device) for i in indxs]
    x2 = [torch.tensor(preprocess_tor_data(pd.DataFrame(dataset.iloc[i.item()]['nucleotide_tor_angles']))
                       .to_numpy()).to(device) for i in indxs]
    src_batch = pad_sequence(x2, padding_value=0).float().to(device)
    tgt_batch = pad_sequence(x1, padding_value=0).float().to(device)

    # x1 = pack_sequence(x1, enforce_sorted=False).float().to(device)
    # x2 = pack_sequence(x2, enforce_sorted=False).float().to(device)
    len_src_batch = torch.tensor([len(x) for x in x2], dtype=torch.int64)
    len_tgt_batch = torch.tensor([len(x) for x in x1], dtype=torch.int64)
    y = torch.tensor([dataset.iloc[i.item()]['rmsd'] for i in indxs]).float().unsqueeze(1).to(device)
    return src_batch, tgt_batch, y, len_src_batch, len_tgt_batch

epochs = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
example_tor_df = preprocess_tor_data(target_tor_angles_example_df)
example_tor_df

,alpha,beta,gamma,delta,epsilon,zeta,chi,name_A,name_C,name_U,name_G
0,0.097000,0.993581,0.120603,0.833796,0.264545,0.677159,0.263321,0,1,0,0
1,0.178395,0.971617,0.094769,0.172924,0.152626,0.723978,0.309414,0,1,0,0
2,0.076171,0.995211,0.444197,0.312134,0.153804,0.734834,0.356509,0,0,0,1
3,0.148753,0.976943,0.007240,0.455361,0.167403,0.794531,0.243671,0,1,0,0
4,0.013597,1.000000,0.740671,0.272939,0.393939,0.697512,0.253647,0,1,0,0
5,0.131048,0.949147,0.700398,0.456675,0.255071,0.687712,0.013760,0,0,0,1
6,0.247264,0.962356,0.000000,0.681897,0.200562,0.769144,0.353827,0,1,0,0
7,0.119056,0.988455,0.336404,0.425890,0.211804,0.756771,0.201038,0,0,0,1
8,0.097000,0.982294,0.617711,0.255294,0.233517,0.799240,0.191463,0,1,0,0
9,0.074416,0.973367,1.000000,0.328728,0.074704,0.820334,0.000000,1,0,0,0


In [39]:
print("Start learning")
train_history, valid_history = [], []
train_batch = 0
enc = Encoder().to(device)
dec = DecoderWithAttention().to(device)
enc_dec = EncoderDecoder(enc, dec).to(device)

lr = 0.001
optimizer = torch.optim.Adam(enc_dec.parameters(), lr=lr)

epochs = 8
batch_size = 128
loss_fn = nn.MSELoss()

for epoch in range(epochs):
    print("Running training loop")
    enc_dec.train()
    batches = shuffle_batches(torch.randperm(train_ds.shape[0]), batch_size, train_ds.shape[0])
    train_losses = []
    p_bar = tqdm(batches, unit='batch')
    for idxs in p_bar:
        p_bar.set_description(f"Epoch: {epoch + 1}")
        optimizer.zero_grad()
        src, tgt, y, src_len, tgt_len = create_batch(train_ds, idxs, device)
        
        outputs = enc_dec(src, tgt, src_len, batch_size)
        # Token START nie jest przewidywany przez dekoder
        tgt = tgt[1:].view(-1)

        # Tablekę outputs zaczęliśmy uzupełniać od indeksu 1
        outputs = outputs[-1].view(-1, 1)

        # print(outputs.shape)
        # print(y.shape)

        loss = loss_fn(outputs, y)
        loss.backward()

        # Ucinanie gradientu
        torch.nn.utils.clip_grad_norm_(enc_dec.parameters(), 2.)
        
        optimizer.step()
        train_losses.append(loss.item())
        train_loss = np.mean(train_losses)
        p_bar.set_postfix(train_loss=train_loss)
        train_history.append(dict(epoch=epoch, batch=train_batch, batch_loss=loss.item()))
        train_batch += 1
    
    print("Running validation loop")
    enc_dec.eval()
    

    with torch.no_grad():
        batches = shuffle_batches(torch.randperm(valid_ds.shape[0]), batch_size, valid_ds.shape[0])
        p_bar = tqdm(batches, unit='batch')
        for idxs in p_bar:
            p_bar.set_description(f"Epoch: {epoch + 1}")
            src, tgt, y, src_len, tgt_len = create_batch(valid_ds, idxs, device)
            y_pred = enc_dec(src, tgt, src_len, batch_size)
            y_pred = y_pred[-1].view(-1, 1)
            valid_loss = loss_fn(y_pred, y).item()
            valid_rmse = ((y_pred - y)**2).mean().sqrt().item()

            p_bar.set_postfix(valid_loss=valid_loss, valid_rmse=valid_rmse)
            valid_history.append(dict(epoch=epoch, train_loss=train_loss, valid_loss=valid_loss, valid_rmse=valid_rmse))
        
    

Start learning
Running training loop


Epoch: 1:   4%|▎         | 9/249 [00:27<12:12,  3.05s/batch, train_loss=118] 


KeyboardInterrupt: 

In [48]:
print("Start testing")
enc_dec.eval()
preds, values = [], []
with torch.no_grad():
    batches = shuffle_batches(torch.randperm(test_ds.shape[0]), batch_size, test_ds.shape[0])
    p_bar = tqdm(batches, unit='batch')
    for batch_idx, idxs in enumerate(p_bar):
        p_bar.set_description(f"Batch: {batch_idx + 1}")
        src, tgt, y, src_len, tgt_len = create_batch(test_ds, idxs, device)
        y_pred = enc_dec(src, tgt, src_len, batch_size)
        y_pred = y_pred[-1].view(-1, 1)
        preds.append(y_pred)
        values.append(y)
print(len(preds), preds[0].shape)
print(len(values), values[0].shape)
print(preds[0][-4:])
print(values[0][-4:])
preds, values = torch.cat(preds).detach().cpu().numpy(), torch.cat(values).detach().cpu().numpy()
test_rmse = np.sqrt(((preds - values) ** 2).mean())

print("Test RMSE:", test_rmse)

Start testing


Batch: 4: 100%|██████████| 4/4 [00:11<00:00,  2.97s/batch]

4 torch.Size([128, 1])
4 torch.Size([128, 1])
tensor([[0.2093],
        [0.2259],
        [0.2258],
        [0.2127]], device='cuda:0')
tensor([[3.2290],
        [4.3010],
        [4.0890],
        [3.4750]], device='cuda:0')
Test RMSE: 3.42314
